In [1]:
import asyncio
from typing import Annotated

from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel import Kernel

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [16]:
chat_history = ChatHistory()
chat_history.add_user_message("I'd like to go to Saigon on December 30, 2025")

In [17]:
class TravelBookingPlugin:
    @kernel_function
    async def book_travel(self, destination: Annotated[str, "Travel destination"], date: Annotated[str, "Travel date"]) -> str:
        # Simulate booking logic
        return f"Travel to {destination} on {date} has been successfully booked!"

In [18]:
kernel = Kernel()

In [19]:
plugin = TravelBookingPlugin()
kernel.add_plugin(plugin, "TravelBookingPlugin")

KernelPlugin(name='TravelBookingPlugin', description=None, functions={'book_travel': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='book_travel', plugin_name='TravelBookingPlugin', description=None, parameters=[KernelParameterMetadata(name='destination', description='Travel destination', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'Travel destination'}, include_in_function_choices=True), KernelParameterMetadata(name='date', description='Travel date', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'Travel date'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_proper

In [20]:
# Retrieve variables from .env
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Define the Azure OpenAI AI Connector
azure_chat_completion = AzureChatCompletion(
    api_key=api_key,
    endpoint=endpoint,
    deployment_name=deployment_name
)

In [21]:
request_settings = AzureChatPromptExecutionSettings(function_choice_behavior=FunctionChoiceBehavior.Auto())


In [22]:
async def main():
    response = await azure_chat_completion.get_chat_message_content(chat_history=chat_history, settings=request_settings, kernel=kernel)
    assert response is not None
    
    """
    User: Book me a flight to Bangkok.
    Model: Sure, I'd love to help you book a flight. Could you please specify the date?
    User: I want to travel on December 30, 2025.
    Model: Your flight to Bangkok on December 30, 2025, has been successfully booked. Safe travels!
    """
    
    print(f"`{response}`")
    # Output: Your flight to Bangkok on December 30, 2025, has been successfully booked. Safe travels!  

    # Add the model's response to our chat history context
    chat_history.add_assistant_message(response.content)
    
import nest_asyncio
nest_asyncio.apply()

await main()

`Your travel to Saigon on December 30, 2025, has been successfully booked! Safe travels!`
